### 1. Import Libraries

In [79]:
import os
import sys
import warnings

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

%matplotlib inline
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

### 2. Import from mlcblab

In [80]:
from mlcvlab.models.nn4 import NN4
from mlcvlab.nn.losses import l2
from mlcvlab.optim.sgd import SGD
# TODO: Import all the necessary code from mlcvlab package as you need... 
from mlcvlab.optim.sync_sgd import sync_sgd
from mlcvlab.optim.async_sgd import async_sgd

import mlcvlab.optim.sync_sgd as sync_sgd
import mlcvlab.optim.async_sgd as async_sgd
import mlcvlab.models.nn4 as nn4


from importlib import reload 



sync_sgd = reload(sync_sgd)
async_sgd = reload(async_sgd)
nn4 = reload(nn4)



### 3. Set Seed

In [81]:
np.random.seed(42)

### 4. Helper functions

In [65]:
def load_dataset():
    x, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
    #TODO

    xu = np.concatenate((np.ones((70000,1),dtype='float64'),x),axis=1)
    return xu,y

def prepare_data(x, y):
    #TODO
    y = y.astype(np.int16)
    y[y // 2 == 0] = 1
    y[y // 2 != 0] = 0
    
    return x, y

def split_train_test(x,y):
    #TODO
    X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 10000,random_state=42) 
    return X_train, X_test, y_train, y_test

def minibatch(X_train,y_train,K):
    #TODO
    # Batch Size: K
    # X_train_batches, y_train_batches should be a list of lists of size K.
    training_data = (X_train, y_train)
    m = len(X_train)   # Number of data points

    X_mini_batches = [training_data[0][k:k+K] for k in range(0,m,K)]

    # this will generate a slice of dataset of size K (X_train)

    y_mini_batches = [training_data[1][k:k+K] for k in range(0,m,K)]

    # this will generate a slice of dataset of size K (y_train)

#    X_train_batches = mini_batches(0)
#    y_train_batches = mini_batches(1)

    return X_mini_batches,y_mini_batches



In [77]:
def initialize_model():
    #TODO (Can use the similar approach used in HW1)
    # e.g. He Initialization for W0-W2, Xavier Initialization for W3
    # Also, initialize your model with a dropout parameter of 0.25 and use_batchnorm being true.

    sizes = [785,200,200,200,1]
    W0 = np.random.randn(200,785)*np.sqrt(1/785)
    W1 = np.random.randn(200,200)*np.sqrt(1/200)
    W2 = np.random.randn(200,200)*np.sqrt(1/200)
    W3 = np.random.randn(1,200)*np.sqrt(1/1)
    print(f"Size of W0 : {W0.shape}, Size of W1 : {W1.shape}, Size of W2 : {W2.shape}, Size of W3 : {W3.shape}")
    four_layer_nn  = nn4.NN4()
    four_layer_nn.layers[0].W = W0
    four_layer_nn.layers[1].W = W1
    four_layer_nn.layers[2].W = W2
    four_layer_nn.layers[3].W = W3

    return four_layer_nn

def train_model(model, X_train_batches, y_train_batches):
    #TODO : Call async_SGD and sync_SGD to train two versions of the same model. Compare their outcomes and runtime.
    #Update both your models with final updated weights and return them
    model_async = sync_sgd.sync_sgd(model,X_train_batches, y_train_batches)
    model_sync = async_sgd.async_sgd(model,X_train_batches,y_train_batches)
    return model_async, model_sync

def test_model(model, X_test, y_test):
    y_pred = model.nn4(X_test)
    y_pred = 1 * (y_pred < 0.5)
    accuracy = np.sum(np.abs(y_pred-y_test) > 0)/ y_pred.shape[1]
    #TODO: Call model.nn4 to test model.
    return accuracy

### 5. Run the program

In [67]:
#load data
x, y = load_dataset()

#prepare data
x, y = prepare_data(x,y)

# split data set
X_train, X_test, y_train, y_test = split_train_test(x,y)


#mini batches
X_train_batches, y_train_batches = minibatch(X_train,y_train,100)

In [82]:
#initialize model
model = initialize_model()

#testing the initial model

initial_model = test_model(model, X_test.transpose(), y_test)
print(f"Initial Model Accuracy : {initial_model}")


#training model
model_async, model_sync = train_model(model, X_train_batches, y_train_batches)
print(f"Completed training, now testing...")   

#testing model
accuracy_async = test_model(model, X_test.transpose(), y_test)
print(f"Completed testing model using asynchronous SGD - Accuracy : {accuracy_async}")   

accuracy_sync = test_model(model, X_test.transpose(), y_test)
print(f"Completed testing model using synchronous SGD - Accuracy : {accuracy_sync}") 

Size of W0 : (200, 785), Size of W1 : (200, 200), Size of W2 : (200, 200), Size of W3 : (1, 200)
Initial Model Accuracy : 0.6756
Completed training for 5 epochs


/Users/rajasunkara/Desktop/CS6406/Assignment/2022-SP-101-102-HW2-rs5cq/mlcvlab/nn/activations.py:16: RuntimeWarning: overflow encountered in exp
  return 1.0/(1+np.exp(-x))


Completed training for 10 epochs
Completed training for 15 epochs
Completed training for 20 epochs
Completed training for 25 epochs
Completed training for 30 epochs
Completed training, now testing...
Completed testing model using asynchronous SGD - Accuracy : 0.8977
Completed testing model using synchronous SGD - Accuracy : 0.8977
